In [99]:
import numpy as np
import pandas as pd
import torch
import sklearn
from tqdm.auto import tqdm
import random
import matplotlib
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import lightgbm as lgb
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import random
import optuna
import lightgbm as lgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
train_data = pd.read_parquet('/kaggle/input/wunderfund-ds/datasets/train.parquet')
valid_data = pd.read_parquet('/kaggle/input/wunderfund-ds/datasets/valid.parquet')

In [3]:
pd.set_option('display.max_columns', None)
train_data.head(15)

,seq_ix,step_in_seq,need_prediction,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,dp0,dp1,dp2,dp3,dv0,dv1,dv2,dv3,t0,t1
0,0,0,0,1.077446,-0.710366,0.373591,0.835558,0.050204,-0.139710,1.367384,-0.771555,0.148147,-1.456244,0.578167,-1.877690,1.100824,0.427610,-0.696487,0.635055,-1.207726,0.696239,0.161046,-0.109626,0.195265,-0.051979,0.309448,0.695902,-0.031078,-0.053527,-0.475214,-0.410787,0.805692,0.516635,-0.368215,-0.503497,-0.564226,1.864018
1,0,1,0,1.100140,-0.562796,0.483658,0.875354,0.138444,-0.010037,1.354714,-0.532586,0.083941,-0.729727,0.569449,-2.096521,1.969383,0.080534,0.001843,0.703261,-0.045569,0.116429,-0.809482,-0.977422,-1.942690,1.960404,-1.614392,0.006176,0.950145,-0.083598,-0.604252,-0.703991,-1.925171,1.252957,-1.563974,-1.929990,-0.239079,2.168843
2,0,2,0,1.100140,-0.562796,0.483658,0.875354,0.138444,-0.010037,1.870266,-0.550814,0.169576,-1.678178,0.638001,-2.411377,1.032822,0.087059,-0.244966,1.113492,0.282883,0.457989,0.559933,-0.819305,-1.900832,-0.404487,1.829278,0.568037,-0.168259,1.632066,-0.178565,0.121240,1.377339,1.803970,-0.402838,1.042579,2.237778,2.459723
3,0,3,0,0.955471,-1.486537,0.230353,0.676854,-0.262686,-0.366873,2.011996,-0.698041,0.080985,-1.911116,0.272987,-1.112860,0.330098,1.132920,-1.037791,0.730749,0.686394,-0.034393,1.135930,-1.775983,-0.879662,1.436832,-0.799631,1.406140,-0.426942,-0.743530,0.680013,1.394857,0.785266,1.424660,1.258192,0.914074,1.998699,2.272444
4,0,4,0,1.025123,-0.832006,0.317650,0.759677,-0.062770,-0.230353,2.174650,-0.729111,0.080828,-1.689394,0.452587,-2.116584,1.355590,0.441056,-0.209733,0.604876,0.065687,0.319750,0.725003,-1.604731,0.280972,0.390781,-0.400034,1.059297,0.195700,0.293986,0.494971,0.267884,-1.315502,1.775372,1.302424,1.355667,1.998699,2.324245
5,0,5,0,1.025123,-0.832006,0.317650,0.759677,-0.062770,-0.230353,2.174650,-0.729111,0.080828,-1.689394,0.452587,-2.116584,0.973252,0.674443,-0.276298,0.616777,0.195214,0.109516,0.255625,-1.115982,0.045688,-0.240554,-0.461170,0.338087,0.195700,0.293986,0.494971,0.267884,-0.837387,0.943961,1.603907,-1.023004,1.998699,2.384014
6,0,6,0,1.513589,-0.893908,0.533636,0.855287,0.002509,-0.184203,2.453884,-0.866762,0.028346,-1.644391,0.612769,-2.262725,-0.254287,0.855977,-0.808914,0.915745,-0.155155,0.168451,0.028348,-0.963426,0.265320,-0.618886,-0.255325,0.118749,0.681039,1.529558,-0.985610,-0.551258,-1.137001,0.848029,1.124470,-1.207640,1.998699,2.547385
7,0,7,0,1.464186,-1.244778,0.462610,0.810910,-0.123259,-0.299230,2.462487,-0.866567,-0.088394,-1.820452,0.554659,-2.177179,1.052781,0.080199,-0.519025,0.517462,-0.077361,0.427447,0.334360,-0.823628,0.292343,-0.461693,-0.179990,0.306699,-0.200818,-0.418644,1.114717,1.562719,0.815302,-0.375368,0.984795,-0.885432,1.998699,2.583247
8,0,8,0,0.975466,-1.207532,0.263985,0.699108,-0.153660,-0.316330,2.183105,-0.729091,0.054416,-1.867551,0.346121,-2.040561,0.842251,0.461688,-0.158974,1.102469,0.035685,0.006719,-0.321405,-0.737453,-0.145020,-0.460459,-0.443208,-0.151436,0.195700,0.293986,0.494971,0.267884,-0.574595,0.749670,0.370902,0.513491,1.998699,2.499569
9,0,9,0,0.939725,-1.926403,0.195930,0.645631,-0.373591,-0.535083,1.375668,-0.770180,-0.169248,-2.402848,0.206814,-1.110828,-0.659417,1.840076,-1.144763,1.071411,-0.476799,0.257170,0.423516,-0.072031,0.645984,-0.549509,0.441215,1.648750,-1.314023,1.278574,-1.805327,-0.492389,-1.204934,1.433097,0.521036,0.978860,1.520541,2.057273


In [4]:
temp_train = train_data[["seq_ix", "step_in_seq", "need_prediction", "t0", "t1"]]
temp_valid = valid_data[["seq_ix", "step_in_seq", "need_prediction", "t0", "t1"]]

temp_valid.shape

(1444000, 5)

In [5]:
temp_train.shape

(10721000, 5)

# выделение нампай массивов для тренировочных и валидационных датасетов для t0 и t1


In [ ]:
# x_list_t0 = []
# y_list_t0 = []
# for i in tqdm(range(10721)):
#     for j in range(3):
#         value = random.randint(1, 899)
#         x_seq = temp_train[temp_train['seq_ix'] == i].reset_index(drop=True).iloc[value:value+100, [3]].to_numpy()
#         y_seq = temp_train[temp_train['seq_ix'] == i].reset_index(drop=True).iloc[value + 100, [3]].to_numpy()
#         x_list_t0.append(x_seq)
#         y_list_t0.append(y_seq)
# x_train_t0 = np.array(x_list_t0)
# y_train_t0 = np.array(y_list_t0)




In [ ]:
# x_list_t1 = []
# y_list_t1 = []
# for i in tqdm(range(10721)):
#     for j in range(3):
#         value = random.randint(1, 899)
#         x_seq = temp_train[temp_train['seq_ix'] == i].reset_index(drop=True).iloc[value:value+100, [3]].to_numpy()
#         y_seq = temp_train[temp_train['seq_ix'] == i].reset_index(drop=True).iloc[value + 100, [3]].to_numpy()
#         x_list_t1.append(x_seq)
#         y_list_t1.append(y_seq)
# x_train_t1 = np.array(x_list_t1)
# y_train_t1 = np.array(y_list_t1)

In [ ]:
# x_valid_list_t1 = []
# y_valid_list_t1 = []
# for i in tqdm(range(1444)):
#     for j in range(3):
#         value = random.randint(1, 899)
#         x_seq = temp_valid[temp_valid['seq_ix'] == i].reset_index(drop=True).iloc[value:value+100, [3]].to_numpy()
#         y_seq = temp_valid[temp_valid['seq_ix'] == i].reset_index(drop=True).iloc[value + 100, [3]].to_numpy()
#         x_valid_list_t1.append(x_seq)
#         y_valid_list_t1.append(y_seq)
# x_valid_t1 = np.array(x_valid_list_t1)
# y_valid_t1 = np.array(y_valid_list_t1)

# x_valid_list_t0 = []
# y_valid_list_t0 = []
# for i in tqdm(range(1444)):
#     for j in range(3):
#         value = random.randint(1, 899)
#         x_seq = temp_valid[temp_valid['seq_ix'] == i].reset_index(drop=True).iloc[value:value+100, [3]].to_numpy()
#         y_seq = temp_valid[temp_valid['seq_ix'] == i].reset_index(drop=True).iloc[value + 100, [3]].to_numpy()
#         x_valid_list_t0.append(x_seq)
#         y_valid_list_t0.append(y_seq)
# x_valid_t0 = np.array(x_valid_list_t0)
# y_valid_t0 = np.array(y_valid_list_t0)


# Обучение модели для t0

In [ ]:
# x_train_t0 = x_train_t0.squeeze()  # Было (32163, 100, 1) -> Станет (32163, 100)
# x_valid_t0 = x_valid_t0.squeeze()  # То же самое для валидации
# x_train_t0

In [ ]:
# model_t0 = CatBoostRegressor(
#     iterations=1000,         # Максимальное число деревьев
#     learning_rate=0.05,      # Скорость обучения (чем меньше, тем точнее, но дольше)
#     depth=6,                 # Глубина дерева (обычно 6-10)
#     eval_metric='RMSE',  # Метрика для отслеживания качества
#     verbose=100              # Выводить отчет каждые 100 итераций
#     # task_type="GPU"        # Раскомментируйте, если есть видеокарта NVIDIA
# )



# model_t0.fit(
#     x_train_t0, y_train_t0,
#     eval_set=(x_valid_t0, y_valid_t0),   # Передаем валидационные данные
#     early_stopping_rounds=100,  # Если качество не растет 50 шагов — стоп (чтобы не переобучиться)
#     plot=True                  # Рисует красивые графики прямо в ноутбуке
# )

In [ ]:
# preds = model_t0.predict(x_valid_t0)
# print("Точность:", mean_squared_error(y_valid_t0, preds))

In [ ]:
# np.sqrt(0.311)

In [ ]:
# print(train_data["t0"].max())
# print(train_data["t0"].min())

# Обучение модели для t1

In [ ]:
# x_train_t1 = x_train_t1.squeeze()
# x_valid_t1 = x_valid_t1.squeeze()
# model_t1 = CatBoostRegressor(
#     iterations=1000,
#     learning_rate=0.05,
#     depth=6,
#     eval_metric='RMSE',
#     verbose=100
# )



# model_t1.fit(
#     x_train_t1, y_train_t1,
#     eval_set=(x_valid_t1, y_valid_t1),
#     early_stopping_rounds=100,
#     plot=True
# )

# проверка метрики таски для того что мы обучили

In [ ]:
# from dataclasses import dataclass

# def weighted_pearson_correlation(y_true: np.ndarray, y_pred: np.ndarray) -> float:
#     """
#     Calculates the Weighted Pearson Correlation Coefficient.

#     This metric emphasizes performance on data points with larger target amplitudes
#     (larger price movements) by using the absolute value of the target as a sample weight.

#     Predictions are clipped to the range [-6, 6] before calculation to prevent
#     outliers from dominating the metric.

#     Args:
#         y_true: Ground truth target values (numpy array).
#         y_pred: Predicted values (numpy array).

#     Returns:
#         float: Weighted Pearson correlation coefficient.
#     """
#     # Clip predictions to valid range [-6, 6]
#     y_pred_clipped = np.clip(y_pred, -6.0, 6.0)

#     # Calculate weights based on target amplitude
#     weights = np.abs(y_true)
#     weights = np.maximum(weights, 1e-8)

#     # Calculate weighted means
#     sum_w = np.sum(weights)
#     if sum_w == 0:
#         return 0.0

#     mean_true = np.sum(y_true * weights) / sum_w
#     mean_pred = np.sum(y_pred_clipped * weights) / sum_w

#     # Calculate weighted deviations
#     dev_true = y_true - mean_true
#     dev_pred = y_pred_clipped - mean_pred

#     # Calculate weighted covariance
#     cov = np.sum(weights * dev_true * dev_pred) / sum_w

#     # Calculate weighted variances
#     var_true = np.sum(weights * dev_true**2) / sum_w
#     var_pred = np.sum(weights * dev_pred**2) / sum_w

#     # Compute correlation
#     if var_true <= 0 or var_pred <= 0:
#         return 0.0

#     corr = cov / (np.sqrt(var_true) * np.sqrt(var_pred))
    
#     return float(corr)


# @dataclass
# class DataPoint:
#     seq_ix: int
#     step_in_seq: int
#     need_prediction: bool
#     #
#     state: np.ndarray


# class PredictionModel:
#     def predict(self, data_point: DataPoint) -> np.ndarray:
#         # return dummy prediction
#         return np.zeros(2)


# class ScorerStepByStep:
#     def __init__(self, dataset_path: str):
#         self.dataset = pd.read_parquet(dataset_path)

#         # Calc feature dimension: first 3 columns are seq_ix, step_in_seq & need_prediction
#         # Total columns: 3 metadata + 32 features + 2 targets = 37
#         # Features are cols [3:35]
#         self.dim = 2
#         self.features = self.dataset.columns[3:35]
#         self.targets = self.dataset.columns[35:]

#     def score(self, model: PredictionModel) -> dict:
#         predictions = []
#         targets = []

#         prediction = None

#         # Iterate over numpy array for speed
#         for row in tqdm(self.dataset.values):
#             seq_ix = row[0]
#             step_in_seq = row[1]
#             need_prediction = row[2]
#             lob_data = row[3:35]  # 32 features
#             labels = row[35:]     # 2 targets
#             #
#             data_point = DataPoint(seq_ix, step_in_seq, need_prediction, lob_data)
#             prediction = model.predict(data_point)

#             self.check_prediction(data_point, prediction)
#             if prediction is not None:
#                 predictions.append(prediction)
#                 targets.append(labels)

#         # report metrics
#         return self.calc_metrics(np.array(predictions), np.array(targets))

#     def check_prediction(self, data_point: DataPoint, prediction: np.ndarray):
#         if not data_point.need_prediction:
#             if prediction is not None:
#                 raise ValueError(f"Prediction is not needed for {data_point}")
#             return

#         if prediction is None:
#             raise ValueError(f"Prediction is required for {data_point}")

#         if prediction.shape[0] != self.dim:
#             raise ValueError(
#                 f"Prediction has wrong shape: {prediction.shape[0]} != {self.dim}"
#             )

#     def calc_metrics(self, predictions: np.ndarray, targets: np.ndarray) -> dict:
#         scores = {}
#         for ix_target, target_name in enumerate(self.targets):
#             scores[target_name] = weighted_pearson_correlation(
#                 targets[:, ix_target], predictions[:, ix_target]
#             )
#         scores["weighted_pearson"] = np.mean(list(scores.values()))
#         return scores

In [ ]:
# y_true = np.column_stack((y_valid_t0, y_valid_t1))
# y_true

In [ ]:
# preds_t0 = model_t0.predict(x_valid_t0)
# preds_t1 = model_t1.predict(x_valid_t1)
# y_pred = np.column_stack((preds_t0, preds_t1))

нихуевый скор

In [ ]:
# print(weighted_pearson_correlation(y_true, y_pred))

# визуализации t0 и t1 и анализ аномалий

In [ ]:
plt.figure(figsize=(10, 6))
train_data['t0'].iloc[10657000:10657999].plot(kind='density', linewidth=2)
plt.title('Плотность распределения')
plt.xlabel('Значения')
plt.grid(True, alpha=0.3)

# Добавляем вертикальную линию на среднем значении
mean_val = train_data['t0'].iloc[10657000:10657999].mean()
plt.axvline(x=mean_val, color='red', linestyle='--', label=f'Среднее: {mean_val:.2f}')
plt.legend()
tqdm(plt.show())

In [ ]:
plt.figure(figsize=(10, 6))
train_data['t1'].iloc[10657000:10657999].plot(kind='density', linewidth=2)
plt.title('Плотность распределения')
plt.xlabel('Значения')
plt.grid(True, alpha=0.3)

# Добавляем вертикальную линию на среднем значении
mean_val = train_data['t1'].iloc[10657000:10657999].mean()
plt.axvline(x=mean_val, color='red', linestyle='--', label=f'Среднее: {mean_val:.2f}')
plt.legend()
tqdm(plt.show())

In [ ]:
trash_holds_t0 = temp_train[(temp_train["t0"] > 2) | (temp_train["t0"] < -2)]
print(trash_holds_t0)

In [ ]:
trash_holds_t0.shape

In [ ]:
temp_train.shape

In [ ]:
trash_holds_t1 = temp_train[(temp_train["t1"] > 5) | (temp_train["t1"] < -5)]

In [ ]:
trash_holds_t1.shape

In [ ]:
trash_holds = temp_train[((temp_train["t1"] > 5) | (temp_train["t1"] < -5)) & ((temp_train["t0"] > 5) | (temp_train["t0"] < -5))]

In [ ]:
trash_holds.shape

In [ ]:
counts = trash_holds["seq_ix"].value_counts()

In [ ]:
print(counts)

In [ ]:
counts = trash_holds_t0["seq_ix"].value_counts()
print(counts)

In [104]:
def weighted_pearson_correlation(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Calculates the Weighted Pearson Correlation Coefficient.

    This metric emphasizes performance on data points with larger target amplitudes
    (larger price movements) by using the absolute value of the target as a sample weight.

    Predictions are clipped to the range [-6, 6] before calculation to prevent
    outliers from dominating the metric.

    Args:
        y_true: Ground truth target values (numpy array).
        y_pred: Predicted values (numpy array).

    Returns:
        float: Weighted Pearson correlation coefficient.
    """
    # Clip predictions to valid range [-6, 6]
    y_pred_clipped = np.clip(y_pred, -6.0, 6.0)

    # Calculate weights based on target amplitude
    weights = np.abs(y_true)
    weights = np.maximum(weights, 1e-8)

    # Calculate weighted means
    sum_w = np.sum(weights)
    if sum_w == 0:
        return 0.0

    mean_true = np.sum(y_true * weights) / sum_w
    mean_pred = np.sum(y_pred_clipped * weights) / sum_w

    # Calculate weighted deviations
    dev_true = y_true - mean_true
    dev_pred = y_pred_clipped - mean_pred

    # Calculate weighted covariance
    cov = np.sum(weights * dev_true * dev_pred) / sum_w

    # Calculate weighted variances
    var_true = np.sum(weights * dev_true**2) / sum_w
    var_pred = np.sum(weights * dev_pred**2) / sum_w

    # Compute correlation
    if var_true <= 0 or var_pred <= 0:
        return 0.0

    corr = cov / (np.sqrt(var_true) * np.sqrt(var_pred))
    
    return float(corr)


def calc_metrics(predictions: np.ndarray, targets: np.ndarray) -> dict:
    scores = {}
    target_names = ['t0', 't1']
    for ix_target, target_name in enumerate(target_names):
        scores[target_name] = weighted_pearson_correlation(
            targets[:, ix_target], predictions[:, ix_target]
        )
    scores["weighted_pearson"] = np.mean(list(scores.values()))
    return scores


# Модель которая по фичам предсказывает t0 и t1

In [7]:
x_train_t0 = train_data.drop(columns=['seq_ix', 'step_in_seq', 'need_prediction', 't0', 't1'])
t0_train = train_data.iloc[:, -2]

In [8]:
x_valid_t0 = valid_data.drop(columns=['seq_ix', 'step_in_seq', 'need_prediction', 't0', 't1'])
t0_val = valid_data.iloc[:, -2]

In [ ]:
# # boost_t0 = CatBoostRegressor(
# #     iterations=100,         # Максимальное число деревьев
# #     learning_rate=0.1,      # Скорость обучения (чем меньше, тем точнее, но дольше)
# #     depth=6,                 # Глубина дерева (обычно 6-10)
# #     eval_metric='RMSE',  # Метрика для отслеживания качества
# #     verbose=100              # Выводить отчет каждые 100 итераций
# # )



# # boost_t0.fit(
# #     x_train_t0, t0_train,
# #     eval_set=(x_valid_t0, t0_val),   # Передаем валидационные данные
# #     early_stopping_rounds=100,  # Если качество не растет 50 шагов — стоп (чтобы не переобучиться)
# #     plot=True                  # Рисует красивые графики прямо в ноутбуке
# # )

print(type(x_train_t0))

## оптюна тюн


In [ ]:



# best_params = None
# best_corr = -10.0

# # Функция для оптимизации
# def objective(trial):
#     # Предлагаемые гиперпараметры
#     params = {
#         'objective': 'regression',
#         'metric': 'rmse',
#         'boosting_type': 'gbdt',
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
#         'verbose': -1,
#         'seed': 42
#     }
    
    
#     # Обучение с ранней остановкой
#     model = lgb.LGBMRegressor(**params)
#     # model = lgb.train(
#     #     params,
#     #     train_data_t0,
#     #     valid_sets=[valid_data_t0],
#     #     callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
#     # )
#     model.fit(x_train_t0, t0_train, eval_set=(x_valid_t0, t0_val))
    
#     # Прогноз и оценка
#     preds = model.predict(x_valid_t0)
#     corr = weighted_pearson_correlation(t0_val, preds)
    


#     return corr

# # Создание study и запуск оптимизации
# study = optuna.create_study(
#     direction='maximize',
#     sampler=optuna.samplers.TPESampler(seed=42),
#     pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
# )

# study.optimize(objective, n_trials=50, timeout=3600, n_jobs=-1)

# # Лучшие параметры
# print("Лучшие параметры:", study.best_params)
# print("Лучшее значение accuracy:", study.best_value)

# # Функция для оптимизации
# def objective(trial):
#     # Предлагаемые гиперпараметры
#     params = {
#         'objective': 'regression',
#         'metric': 'rmse',
#         'boosting_type': 'gbdt',
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
#         'verbose': -1,
#         'seed': 42
#     }
    
#     # {'learning_rate': 0.04657440386654163, 'feature_fraction': 0.5137930055660633, 'bagging_fraction': 0.7146977160454794, 'bagging_freq': 9}
#     # Обучение с ранней остановкой
#     model = lgb.LGBMRegressor(**params)
#     # model = lgb.train(
#     #     params,
#     #     train_data_t0,
#     #     valid_sets=[valid_data_t0],
#     #     callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
#     # )
#     model.fit(x_train_t1, t1_train, eval_set=(x_valid_t1, t1_val))
    
#     # Прогноз и оценка
#     preds = model.predict(x_valid_t1)
#     corr = weighted_pearson_correlation(t1_val, preds)
    


#     return corr

# # Создание study и запуск оптимизации
# study = optuna.create_study(
#     direction='maximize',
#     sampler=optuna.samplers.TPESampler(seed=42),
#     pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
# )

# study.optimize(objective, n_trials=50, timeout=3600, n_jobs=-1)

# # Лучшие параметры
# print("Лучшие параметры:", study.best_params)
# print("Лучшее значение accuracy:", study.best_value)

In [9]:

# Создание Dataset объектов
train_data_t0 = lgb.Dataset(x_train_t0, label=t0_train)
valid_data_t0 = lgb.Dataset(x_valid_t0, label=t0_val, reference=train_data_t0)

# Параметры для регрессии
params = {
    'objective': 'regression',  # основная задача - регрессия
    'metric': 'rmse',          # метрика для оценки
    'boosting_type': 'gbdt',   # тип бустинга
    'num_leaves': 31,          # максимальное количество листьев в дереве
    'learning_rate': 0.2046061119024124, 
    'feature_fraction': 0.6261250790392358, 
    'bagging_fraction': 0.92904133638405, 
    'bagging_freq': 7,
    'verbose': 0,              # отключение вывода
    'seed': 42                 # для воспроизводимости
}

# Обучение модели
boost_t0 = lgb.train(
    params,
    train_data_t0,
    num_boost_round=1000,           # максимальное количество деревьев
    valid_sets=[valid_data_t0],        # валидационный набор
    callbacks=[
        lgb.early_stopping(100),     # ранняя остановка
        lgb.log_evaluation(100)     # вывод лога каждые 100 итераций
    ]
)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.20943
[200]	valid_0's rmse: 1.2091
Early stopping, best iteration is:
[185]	valid_0's rmse: 1.20897


In [10]:
x_train_t1 = train_data.drop(columns=['seq_ix', 'step_in_seq', 'need_prediction', 't0', 't1'])
t1_train = train_data.iloc[:, -1]

x_valid_t1 = valid_data.drop(columns=['seq_ix', 'step_in_seq', 'need_prediction', 't0', 't1'])
t1_val = valid_data.iloc[:, -1]

In [11]:
# boost_t1 = CatBoostRegressor(
#     iterations=1000,         # Максимальное число деревьев
#     learning_rate=0.1,      # Скорость обучения (чем меньше, тем точнее, но дольше)
#     depth=9,                 # Глубина дерева (обычно 6-10)
#     eval_metric='RMSE',  # Метрика для отслеживания качества
#     verbose=150              # Выводить отчет каждые 100 итераций
# )



# boost_t1.fit(
#     x_train_t1, t1_train,
#     eval_set=(x_valid_t1, t1_val),   # Передаем валидационные данные
#     early_stopping_rounds=100,  # Если качество не растет 50 шагов — стоп (чтобы не переобучиться)
#     plot=True                  # Рисует красивые графики прямо в ноутбуке
# )

# Создание Dataset объектов
train_data_t1 = lgb.Dataset(x_train_t1, label=t1_train)
valid_data_t1 = lgb.Dataset(x_valid_t1, label=t1_val, reference=train_data_t1)

# Параметры для регрессии
params = {
    'objective': 'regression',  # основная задача - регрессия
    'metric': 'rmse',          # метрика для оценки
    'boosting_type': 'gbdt',   # тип бустинга
    'num_leaves': 31,          # максимальное количество листьев в дереве
    'learning_rate': 0.04657440386654163, 
    'feature_fraction': 0.5137930055660633, 
    'bagging_fraction': 0.7146977160454794, 
    'bagging_freq': 9,
    'verbose': 0,              # отключение вывода
    'seed': 42                 # для воспроизводимости
}

# Обучение модели
boost_t1 = lgb.train(
    params,
    train_data_t1,
    num_boost_round=1000,           # максимальное количество деревьев
    valid_sets=[valid_data_t1],        # валидационный набор
    callbacks=[
        lgb.early_stopping(100),     # ранняя остановка
        lgb.log_evaluation(100)     # вывод лога каждые 100 итераций
    ]
)


Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.97859
[200]	valid_0's rmse: 1.9784
Early stopping, best iteration is:
[180]	valid_0's rmse: 1.97834


In [12]:
pred_t0 = boost_t0.predict(x_valid_t0)
pred_t1 = boost_t1.predict(x_valid_t1)
y_predict = np.column_stack((pred_t0, pred_t1))
y_predict_rofl = np.column_stack((pred_t0, np.zeros(len(pred_t0))))

In [13]:
calc_metrics(y_predict, np.column_stack((t0_val, t1_val)))

{'t0': 0.32194055171619734,
 't1': 0.09129282074250476,
 'weighted_pearson': np.float64(0.20661668622935106)}

In [14]:
from joblib import dump
dump(boost_t0, 't0_model.joblib')
dump(boost_t1, 't1_model.joblib')

['t1_model.joblib']

In [ ]:
train_data[['t0', 't1']].corr()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(18, 16))  # Увеличиваем размер фигуры
corr_matrix = train_data.corr()
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Heatmap of DataFrame', fontsize=20, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
random_values = random.sample(list(range(10000)), 2000)
random_values

In [ ]:
new_train = train_data.drop(columns=['seq_ix', 't0', 't1', 'step_in_seq', 'need_prediction'])
train_x = []
for i in random_values:
    z = new_train.iloc[i * 1000 : i * 1000 + 100, :].to_numpy()
    train_x.append(z)
train_x = np.array(train_x)

In [ ]:
print(train_x.shape)
print(train_x)

In [ ]:
new_train_y = train_data.iloc[:, -2:]
print(new_train_y)


In [ ]:
train_y = []
for i in random_values:
    z = new_train_y.iloc[i * 1000: i * 1000 + 100, :].to_numpy()
    train_y.append(z)
train_y = np.array(train_y)
print(train_y)

In [ ]:
train_data[train_data['seq_ix'] == 5726]

In [ ]:
train_y.shape


In [ ]:
valid_values = random.sample(list(range(1400)), 700)
new_val_x = valid_data.drop(columns=['seq_ix', 't0', 't1', 'step_in_seq', 'need_prediction'])
valid_x = []
for i in valid_values:
    z = new_val_x.iloc[i * 1000 : i * 1000 + 100, :].to_numpy()
    valid_x.append(z)
valid_x = np.array(valid_x)

new_val_y = valid_data.iloc[:, -2:]
valid_y = []
for i in valid_values:
    z = new_val_y.iloc[i * 1000 : i * 1000 + 100, :].to_numpy()
    valid_y.append(z)
valid_y = np.array(valid_y)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.nn import Linear, Conv2d, ReLU, Softmax, MaxPool2d, Flatten, Sequential

In [ ]:
train_x.shape

In [ ]:
# Преобразуем в тензоры PyTorch
train_x_tensor = torch.from_numpy(train_x).unsqueeze(1)  # Добавляем канал: (2000, 1, 100, 32)
train_y_tensor = torch.from_numpy(train_y)

print(f"Размерность входных данных: {train_x_tensor.shape}")  # torch.Size([2000, 1, 100, 32])
print(f"Размерность целевых данных: {train_y_tensor.shape}")  # torch.Size([2000, 2])

# Создаем Dataset
train_dataset = TensorDataset(train_x_tensor, train_y_tensor)

In [ ]:
val_x_tensor = torch.from_numpy(valid_x).unsqueeze(1)
val_y_tensor = torch.from_numpy(valid_y)
valid_dataset = TensorDataset(val_x_tensor, val_y_tensor)

In [ ]:
train_data.mean()

In [ ]:
train_data.std()

In [ ]:
print("huy")

# предсказываем тройками

In [3]:
train_data.head(5)

,seq_ix,step_in_seq,need_prediction,p0,p1,p2,p3,p4,p5,p6,...,dp0,dp1,dp2,dp3,dv0,dv1,dv2,dv3,t0,t1
0,0,0,0,1.077446,-0.710366,0.373591,0.835558,0.050204,-0.139710,1.367384,...,-0.031078,-0.053527,-0.475214,-0.410787,0.805692,0.516635,-0.368215,-0.503497,-0.564226,1.864018
1,0,1,0,1.100140,-0.562796,0.483658,0.875354,0.138444,-0.010037,1.354714,...,0.950145,-0.083598,-0.604252,-0.703991,-1.925171,1.252957,-1.563974,-1.929990,-0.239079,2.168843
2,0,2,0,1.100140,-0.562796,0.483658,0.875354,0.138444,-0.010037,1.870266,...,-0.168259,1.632066,-0.178565,0.121240,1.377339,1.803970,-0.402838,1.042579,2.237778,2.459723
3,0,3,0,0.955471,-1.486537,0.230353,0.676854,-0.262686,-0.366873,2.011996,...,-0.426942,-0.743530,0.680013,1.394857,0.785266,1.424660,1.258192,0.914074,1.998699,2.272444
4,0,4,0,1.025123,-0.832006,0.317650,0.759677,-0.062770,-0.230353,2.174650,...,0.195700,0.293986,0.494971,0.267884,-1.315502,1.775372,1.302424,1.355667,1.998699,2.324245


In [213]:

three_model_train_data_y_t0


,t0
0,-0.564226
1,-0.239079
2,2.237778
3,1.998699
4,1.998699
...,...
10720995,0.086068
10720996,0.086068
10720997,0.086068
10720998,0.086068


In [249]:
train_data.shape

(10721000, 37)

In [237]:
three_model_train_data = train_data.drop(columns=['seq_ix', 'step_in_seq', 'need_prediction', 't0', 't1'])
three_model_train_data_y_t0 = train_data[['t0']]
three_model_train_data_y_t1 = train_data[['t1']]
x_list = []
y_list_t0 = np.empty(0)
y_list_t1 = np.empty(0)
l = []
random_stocks = random.sample(range(10721), 10000)
for stock in random_stocks:
    random_values = random.sample(range(stock * 1000 + 4, stock * 1000 + 999), 50)
    l.append(random_values)
    y_t0 = three_model_train_data_y_t0.iloc[random_values, :].to_numpy()
    y_t0 = y_t0.squeeze()
    y_list_t0 = np.concatenate([y_list_t0, y_t0])
    y_t1 = three_model_train_data_y_t1.iloc[random_values, :].to_numpy()
    y_t1 = y_t1.squeeze()
    y_list_t1 = np.concatenate([y_list_t1, y_t1])
    for value in random_values:
        x = three_model_train_data.iloc[value - 2:value + 1, :].to_numpy()
        x = x.flatten()
        x_list.append(x)
x_list = np.array(x_list)

In [250]:
import random
import numpy as np

# Предполагаем, что train_data уже загружен
three_model_train_data = train_data.drop(columns=['seq_ix', 'step_in_seq', 'need_prediction', 't0', 't1'])
three_model_train_data_y_t0 = train_data['t0']
three_model_train_data_y_t1 = train_data['t1']

x_list = []
y_list_t0 = []
y_list_t1 = []

random_stocks = random.sample(range(10721), 10000)

for stock in random_stocks:
    # Генерируем 50 случайных индексов для текущего стока
    # Диапазон: от stock*1000+2 до stock*1000+999 (включительно), чтобы можно было взять value-2, value-1, value
    # Но в random.sample нужно передать последовательность, поэтому используем range
    # Примечание: range(stock*1000+2, stock*1000+1000) даст нам индексы от stock*1000+2 до stock*1000+999
    random_values = random.sample(range(stock*1000+2, stock*1000+1000), 50)
    
    for value in random_values:
        # Извлекаем 3 строки фичей (value-2, value-1, value)
        x = three_model_train_data.iloc[value-2:value+1, :].to_numpy()  # форма (3, 32)
        x = x.flatten()  # преобразуем в 1D массив (96,)
        x_list.append(x)
        
        # Извлекаем t0 и t1 для текущего value
        y_t0 = three_model_train_data_y_t0.iloc[value]
        y_t1 = three_model_train_data_y_t1.iloc[value]
        y_list_t0.append(y_t0)
        y_list_t1.append(y_t1)

# Преобразуем списки в массивы
x_list = np.array(x_list)  # форма (500000, 96)
y_list_t0 = np.array(y_list_t0)  # форма (500000,)
y_list_t1 = np.array(y_list_t1)  # форма (500000,)

In [217]:
l[:2]

[[10619098,
  10619992,
  10619980,
  10619086,
  10619362,
  10619424,
  10619412,
  10619882,
  10619038,
  10619888,
  10619337,
  10619615,
  10619639,
  10619933,
  10619967],
 [695457,
  695335,
  695431,
  695597,
  695940,
  695953,
  695600,
  695684,
  695820,
  695098,
  695082,
  695463,
  695517,
  695441,
  695224]]

In [218]:
y_list_t0[9]

np.float64(-0.39208927008285366)

In [219]:
x_list[9]

array([ 1.53769125e+00, -7.18461996e-01,  5.70160906e-01,  8.88309345e-01,
        6.15126366e-02, -1.12804274e-01,  3.09618384e+00, -1.82758584e+00,
        2.17524696e-01, -1.91477666e+00,  6.70533926e-01, -2.34514216e+00,
        9.14048748e-01,  7.22230249e-01, -7.28679204e-01,  9.42618814e-01,
       -1.40171268e-01,  7.96850439e-02,  1.11145049e+00, -2.16333056e-01,
       -7.40484918e-02, -9.24857804e-01,  2.90784982e-01,  1.31727486e+00,
       -1.10860922e+00,  1.84090910e+00,  6.15435754e-02,  6.19694005e-01,
        5.76820993e-01,  1.64638825e+00, -7.13111458e-01,  7.09260362e-01,
        2.59651519e+00, -2.70537606e+00,  4.51541753e-01,  6.61480546e-01,
       -6.73687346e-01, -9.04819748e-01,  2.71297755e+00, -1.10404799e+00,
       -7.32491588e-01, -2.51186174e+00,  2.59191994e-01, -6.76259757e-01,
        7.96309416e-01,  3.47743325e-01, -7.54091695e-01,  9.84283924e-01,
       -1.06333281e-01, -2.89966726e-01, -1.78251760e+00, -9.11272174e-01,
       -1.43834716e+00, -

In [220]:
sample = train_data.iloc[736324 - 2:736324 + 2, :]
sample

,seq_ix,step_in_seq,need_prediction,p0,p1,p2,p3,p4,p5,p6,...,dp0,dp1,dp2,dp3,dv0,dv1,dv2,dv3,t0,t1
736322,736,322,1,1.025123,-0.832006,0.317650,0.759677,-0.062770,-0.230353,1.218002,...,0.802226,-0.667419,0.254902,-0.120731,0.144780,-0.667419,0.610484,-0.833781,-0.239079,0.519200
736323,736,323,1,1.513589,-0.893908,0.533636,0.855287,0.002509,-0.184203,1.584354,...,0.785036,-1.037937,0.653377,-0.506348,-0.181652,-0.181652,-0.181652,-0.181652,-0.239079,0.650693
736324,736,324,1,1.464186,-1.244778,0.462610,0.810910,-0.123259,-0.299230,1.649237,...,0.301855,0.482248,-1.118711,-1.664082,-1.446104,0.761353,-1.108675,-1.404533,-0.239079,0.664639
736325,736,325,1,1.537691,-0.718462,0.570161,0.888309,0.061513,-0.112804,1.562721,...,0.695908,-0.636056,-0.712852,-0.715884,1.012808,0.761945,-0.466803,-0.777044,1.042384,0.921649


In [238]:
three_model_valid_data = valid_data.drop(columns=['seq_ix', 'step_in_seq', 'need_prediction', 't0', 't1'])
three_model_valid_data_y_t0 = valid_data[['t0']]
three_model_valid_data_y_t1 = valid_data[['t1']]
val = []
x_list_valid = []
y_list_t0_valid = np.empty(0)
y_list_t1_valid = np.empty(0)
random_stocks_valid = random.sample(range(1444), 1000)
for stock in random_stocks_valid:
    random_values = random.sample(range(stock * 1000 + 4, stock * 1000 + 999), 15)
    val.append(random_values)
    y_t0 = three_model_valid_data_y_t0.iloc[random_values, :].to_numpy()
    y_t0 = y_t0.squeeze()
    y_list_t0_valid = np.concatenate([y_list_t0_valid, y_t0])
    y_t1 = three_model_valid_data_y_t1.iloc[random_values, :].to_numpy()
    y_t1 = y_t1.squeeze()
    y_list_t1_valid = np.concatenate([y_list_t1_valid, y_t1])
    for value in random_values:
        x = three_model_valid_data.iloc[value - 2:value + 1, :].to_numpy()
        x = x.flatten()
        x_list_valid.append(x)
x_list_valid = np.array(x_list_valid)

In [252]:
import random
import numpy as np

three_model_valid_data = valid_data.drop(columns=['seq_ix', 'step_in_seq', 'need_prediction', 't0', 't1'])
three_model_valid_data_y_t0 = valid_data[['t0']]
three_model_valid_data_y_t1 = valid_data[['t1']]

x_list_valid = []
y_list_t0_valid = []
y_list_t1_valid = []

random_stocks_valid = random.sample(range(1444), 1000)

for stock in random_stocks_valid:
    # Генерируем 50 случайных индексов для текущего стока
    # Диапазон: от stock*1000+2 до stock*1000+999 (включительно), чтобы можно было взять value-2, value-1, value
    # Но в random.sample нужно передать последовательность, поэтому используем range
    # Примечание: range(stock*1000+2, stock*1000+1000) даст нам индексы от stock*1000+2 до stock*1000+999
    random_values_valid = random.sample(range(stock*1000+2, stock*1000+1000), 50)
    
    for value in random_values_valid:
        # Извлекаем 3 строки фичей (value-2, value-1, value)
        x = three_model_valid_data.iloc[value-2:value+1, :].to_numpy()  # форма (3, 32)
        x = x.flatten()  # преобразуем в 1D массив (96,)
        x_list_valid.append(x)
        
        # Извлекаем t0 и t1 для текущего value
        y_t0 = three_model_valid_data_y_t0.iloc[value]
        y_t1 = three_model_valid_data_y_t1.iloc[value]
        y_list_t0_valid.append(y_t0)
        y_list_t1_valid.append(y_t1)

# Преобразуем списки в массивы
x_list_valid = np.array(x_list_valid)  # форма (500000, 96)
y_list_t0_valid = np.array(y_list_t0_valid)  # форма (500000,)
y_list_t1_valid = np.array(y_list_t1_valid)  # форма (500000,)

In [222]:
val[:2]

[[817970,
  817059,
  817400,
  817205,
  817540,
  817293,
  817580,
  817120,
  817102,
  817147,
  817542,
  817926,
  817377,
  817311,
  817651],
 [400982,
  400144,
  400028,
  400662,
  400708,
  400867,
  400850,
  400243,
  400667,
  400453,
  400855,
  400105,
  400373,
  400313,
  400196]]

In [239]:
y_list_t0_valid[0]

np.float64(-0.5638324835274419)

In [240]:
x_list_valid[0]

array([ 1.70855476, -1.57637385,  0.41758139,  0.96857679, -0.26832505,
       -0.19555851,  1.11170208, -0.4307273 ,  0.06025574, -0.7987689 ,
        0.38978343, -0.7697614 ,  1.45938365,  0.21652353, -0.30928479,
        0.55193443, -0.34562613,  0.64334478, -0.20190428, -0.58814572,
       -0.18860977, -0.67326849, -0.00528795,  0.67450334,  0.08415533,
       -0.44591914,  0.65804298, -0.38302436, -0.16382378, -0.16382378,
       -0.16382378, -0.16382378,  1.4418168 , -0.45364661,  0.56557405,
        0.98546243,  0.14321161, -0.11964038,  1.53480196, -0.62109893,
        0.19950978, -0.49497096,  0.65731073, -1.48673644,  0.17824819,
        0.75952968, -0.48684903,  1.47472762, -0.22295118,  0.22567893,
        0.66479312, -0.59354071,  0.45615851, -0.70489377,  0.46689237,
        0.96900514,  1.65528396,  0.38338811, -0.66198276, -1.60859243,
       -1.60221032,  0.93003618, -1.23935188, -1.55848163,  0.99897155,
       -1.20979421,  0.21879797,  0.79300649,  0.04755036, -0.13

In [241]:
sample_valid = valid_data.iloc[817970 - 2:817970 + 2, :]
sample_valid

,seq_ix,step_in_seq,need_prediction,p0,p1,p2,p3,p4,p5,p6,...,dp0,dp1,dp2,dp3,dv0,dv1,dv2,dv3,t0,t1
817968,817,968,1,1.976943,-1.554270,0.561327,0.844485,-0.172731,-0.338834,2.074445,...,-1.006211,-2.125624,1.358163,0.922330,0.482248,0.132113,0.048948,0.076604,0.325147,-0.281715
817969,817,969,1,0.939725,-1.926403,0.195930,0.645631,-0.373591,-0.535083,1.340200,...,-2.432034,1.063739,1.133763,0.302406,-1.463787,2.014479,0.282266,1.812156,0.086068,-0.739949
817970,817,970,1,1.742142,-1.734640,0.236799,0.894073,-0.336178,-0.648386,1.315958,...,-0.958006,0.107226,0.734046,0.721463,-1.272065,1.671804,1.674968,0.890703,-0.153010,-0.967074
817971,817,971,1,1.772124,-1.220640,0.344200,0.927298,-0.173568,-0.401559,1.218002,...,0.283207,0.920577,-0.444534,-0.214750,0.036828,-1.361622,1.594105,-1.796247,-0.153010,-0.971058


In [242]:
print(x_list.shape)

(500000, 96)


In [243]:
print(y_list_t0.shape)
print(y_list_t1.shape)

(500000,)
(500000,)


(500000, 1)
(15000, 96)
(15000, 1)


In [253]:
# Создание Dataset объектов
three_train_data_t0 = lgb.Dataset(x_list, label=y_list_t0)
three_valid_data_t0 = lgb.Dataset(x_list_valid, label=y_list_t0_valid, reference=three_train_data_t0)

# Параметры для регрессии
params = {
    'objective': 'regression',      # Задача регрессии
    'metric': 'rmse',               # Метрика оценки - RMSE
    'boosting_type': 'gbdt',        # Тип бустинга
    'num_leaves': 31,               # Количество листьев
    'learning_rate': 0.05,          # Скорость обучения
    'feature_fraction': 0.9,        # Доля признаков для каждой итерации
    'bagging_fraction': 0.8,        # Доля данных для бэггинга
    'bagging_freq': 5,              # Частота бэггинга
    'verbose': -1,                  # Отключение вывода (0 - базовый, -1 - полное)
    'seed': 42      
}

# Обучение модели
three_boost_t0 = lgb.train(
    params,
    three_train_data_t0,
    num_boost_round=1000,           # максимальное количество деревьев
    valid_sets=[three_valid_data_t0],        # валидационный набор
    callbacks=[
        lgb.early_stopping(100),     # ранняя остановка
        lgb.log_evaluation(100)     # вывод лога каждые 100 итераций
    ]
)

/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:374: UserWarning: Converting column-vector to 1d array
  _log_warning("Converting column-vector to 1d array")


Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.22214
[200]	valid_0's rmse: 1.21804
[300]	valid_0's rmse: 1.21768
Early stopping, best iteration is:
[296]	valid_0's rmse: 1.21765


In [ ]:
pred_t0 = three_boost_t0.predict(x_list_valid)
print(weighted_pearson_correlation(pred_t0, y_list_t0_valid))

In [247]:
# Создание Dataset объектов
three_train_data_t1 = lgb.Dataset(x_list, label=y_list_t1)
three_valid_data_t1 = lgb.Dataset(x_list_valid, label=y_list_t1_valid, reference=three_train_data_t1)

# Параметры для регрессии
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'verbose': -1,
    'seed': 42
}

# Обучение модели
three_boost_t1 = lgb.train(
    params,
    three_train_data_t1,
    num_boost_round=1000,           # максимальное количество деревьев
    valid_sets=[three_valid_data_t1],        # валидационный набор
    callbacks=[
        lgb.early_stopping(100),     # ранняя остановка
        lgb.log_evaluation(100)     # вывод лога каждые 100 итераций
    ]
)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.97988
[200]	valid_0's rmse: 1.97541
[300]	valid_0's rmse: 1.97725
Early stopping, best iteration is:
[201]	valid_0's rmse: 1.97528


In [248]:
pred_t1 = three_boost_t1.predict(x_list_valid)
print(weighted_pearson_correlation(pred_t1, y_list_t1_valid))

-6.949083197174511e-16


In [233]:
# # Функция для оптимизации
# def objective(trial):
#     # Предлагаемые гиперпараметры
#     params = {
#         'objective': 'regression',
#         'metric': 'rmse',
#         'boosting_type': 'gbdt',
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
#         'verbose': -1,
#         'seed': 42
#     }
    
#     # {'learning_rate': 0.04657440386654163, 'feature_fraction': 0.5137930055660633, 'bagging_fraction': 0.7146977160454794, 'bagging_freq': 9}
#     # Обучение с ранней остановкой
#     model = lgb.LGBMRegressor(**params)
#     # model = lgb.train(
#     #     params,
#     #     train_data_t0,
#     #     valid_sets=[valid_data_t0],
#     #     callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
#     # )
#     model.fit(x_train_t1, t1_train, eval_set=(x_valid_t1, t1_val))
    
#     # Прогноз и оценка
#     preds = model.predict(x_valid_t1)
#     corr = weighted_pearson_correlation(t1_val, preds)
    


#     return corr

# # Создание study и запуск оптимизации
# study = optuna.create_study(
#     direction='maximize',
#     sampler=optuna.samplers.TPESampler(seed=42),
#     pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
# )

# study.optimize(objective, n_trials=50, timeout=3600, n_jobs=-1)

# # Лучшие параметры
# print("Лучшие параметры:", study.best_params)
# print("Лучшее значение accuracy:", study.best_value)